In [5]:
def vstack_image(images=[]):
    out = images[0].astype(np.uint8)
    for image in images[1:]:
        out = cv2.vconcat((out, image.astype(np.uint8)))
    return out

def hstack_image(images=[]):
    out = images[0].astype(np.uint8)
    for image in images[1:]:
        out = cv2.hconcat((out, image.astype(np.uint8)))
    return out
    
def resize_image(image, fixed=False):
    return cv2.resize(image, (int(image.shape[1] * SCALE_FACTOR), int(image.shape[0] * SCALE_FACTOR)), interpolation=cv2.INTER_AREA)

In [7]:
import math
import numpy as np
import cv2
import time
from datetime import datetime
import mediapipe as mp
import pandas as pd
import pprint
# from threading import Thread
import multiprocess

ENABLE = True
SCALE_FACTOR = 0.5

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def main():
    global image
    cap = cv2.VideoCapture("/Users/calvin/Documents/NUIG/Thesis/Movies/bowing/bowing8.MP4")
    with mp_pose.Pose(
        min_detection_confidence=0.5,
        min_tracking_confidence=0.5) as pose:
        pose.enable_segmentation = False
        while cap.isOpened():
            success, image = cap.read()
            if not success:
                print("Ignoring emtpty camera frame.")
                break

            image = resize_image(image)

            if ENABLE:
                image.flags.writeable = False
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                results = pose.process(image)

                image.flags.writeable = True
                image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
                mp_drawing.draw_landmarks(
                    image,
                    results.pose_landmarks,
                    mp_pose.POSE_CONNECTIONS,
                    landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())

                keyPoints = storeKeyPoints(results, image.shape)
                extractKeyPoints(image, keyPoints)
                
            cv2.imshow("stack", cv2.flip(image, 1))
            if cv2.waitKey(1) & (0xFF == 27 | 0xFF == ord("q")):
                cv2.destroyAllWindows()
                break
        cap.release()
    
def drawMap(stencil, _keyPoints, kp, ret):
    _keyPoints = find_angleDistance(_keyPoints, image, [(0 , i) for i in range(33)], str(kp))
    stencilN = drawLines(stencil.copy(), _keyPoints, [(kp, i) for i in range(33)])
    stencilN = drawCircles(stencilN, _keyPoints, [i for i in range(33)], small=True, otherPoints="m" + str(kp))
    ret[kp] = stencilN

    
def extractKeyPoints(image, keyPoints):
    stencil  = drawStencil(keyPoints, image.shape)
    stencil1 = drawLines(stencil.copy(), keyPoints, list(mp_pose.POSE_CONNECTIONS))
    
    # sthread2 = CustomThread(drawMap, (stencil.copy(), keyPoints.copy(),  0))
    # sthread3 = CustomThread(drawMap, (stencil.copy(), keyPoints.copy(), 15))
    # sthread4 = CustomThread(drawMap, (stencil.copy(), keyPoints.copy(), 16))
    # sthread5 = CustomThread(drawMap, (stencil.copy(), keyPoints.copy(), 33))
    
    manager = multiprocessing.Manager()
    ret = manager.dict()

    sthread2 = multiprocessing.Process(target=drawMap, args=(stencil.copy(), keyPoints.copy(),  0, ret))
    sthread3 = multiprocessing.Process(target=drawMap, args=(stencil.copy(), keyPoints.copy(), 15, ret))
    sthread4 = multiprocessing.Process(target=drawMap, args=(stencil.copy(), keyPoints.copy(), 16, ret))
    sthread5 = multiprocessing.Process(target=drawMap, args=(stencil.copy(), keyPoints.copy(), 33, ret))
    
    
    sthread2.start()
    sthread3.start()
    sthread4.start()
    sthread5.start()
    
    sthread2.join()
    sthread3.join()
    sthread4.join()
    sthread5.join()
    
#     stencil2 = sthread2.ret
#     stencil3 = sthread3.ret
#     stencil4 = sthread4.ret
#     stencil5 = sthread5.ret

#     vstack1 = vstack_image([stencil3, stencil2]) 
#     vstack2 = vstack_image([image, stencil1])
#     vstack3 = vstack_image([stencil4, stencil5])
#     hstack1 = hstack_image([vstack1, vstack2, vstack3])
    
#     return hstack1

ModuleNotFoundError: No module named 'multiprocess'

In [3]:
_empty = None



# class CustomThread(Thread):
#     def __init__(self, func, args):
#         Thread.__init__(self)
#         self.func = func
#         self.args = args
 
#     def run(self):
#         self.ret = self.func(*self.args)

    
def storeKeyPoints(results, shape):
    keyPoints = {}
    for idx, lm in enumerate(results.pose_landmarks.landmark):
        keyPoints[idx] = { "x" : int(lm.x * shape[1]),
                           "y" : int(lm.y * shape[0]),
                           "z" : round(lm.z, 3),
                           "v" : round(lm.visibility, 3)}
    keyPoints[33] = {"x" : (keyPoints.get(24).get("x", _empty) + keyPoints.get(23).get("x")) // 2,
                     "y" : (keyPoints.get(24).get("y", _empty) + keyPoints.get(23).get("y")) // 2}
    return keyPoints


def drawStencil(keyPoints, imageDimension):
    blankImage = np.zeros(imageDimension)
    image = drawCircles(blankImage, keyPoints, range(33))
    return image
    

def drawCircles(image, keyPoints, circle_points=[], small=False, otherPoints=""):
    for points in circle_points:
        if otherPoints != "":
            _temp = keyPoints[points][otherPoints]
        elif isinstance(keyPoints, dict):
            _temp = (keyPoints[points]["x"], keyPoints[points]["y"])
        elif isinstance(keyPoints, tuple):
            _temp = keyPoints
        if small:
            image = cv2.circle(image, _temp, radius=1, color=(255, 255, 255), thickness=2)     
        else:
            image = cv2.circle(image, _temp, radius=5, color=(0, 0, 255), thickness=-10)
            image = cv2.circle(image, _temp, radius=7, color=(255, 255, 255), thickness=2)   
    return image
    
def drawLines(image, keyPoints, line_points=[]):
    for line in line_points:
        line_start = keyPoints.get(line[0], _empty)
        line_start = (line_start["x"], line_start["y"])
        line_end = keyPoints.get(line[1], _empty)
        line_end = (line_end["x"], line_end["y"])
        image = cv2.line(image, line_start, line_end, color=(255, 255, 255), thickness=1, lineType=cv2.LINE_AA)
    return image

def find_angleDistance(keyPoints, image, points,  reference="0", annotate=True):
    for point in points:
        p1, p2 = point
        if p1 in keyPoints.keys() and p2 in keyPoints.keys():
            p1 = keyPoints.get(p1)
            p2 = keyPoints.get(p2)
            y2y1 = p2.get("y") - p1.get("y")
            x2x1 = p2.get("x") - p1.get("x")
            x2x1 = 1 if x2x1 == 0 else x2x1
            keyPoints[point[1]]["a"+reference] = math.degrees(math.atan(y2y1/x2x1))
            keyPoints[point[1]]["d"+reference] = int(math.sqrt((y2y1**2) + (x2x1**2)))
            keyPoints[point[1]]["m"+reference] = ((p2.get("x") + p1.get("x"))//2, (p2.get("y") + p1.get("y"))//2)
    return keyPoints
    
def annotateMeasurements(image, measurements):
    for measure in measurements:
        if measure.get("midpoint", None):
            x, y = measure.get("midpoint")
            image = cv2.putText(image, str(measure.get("distance", "null")), (x, y), cv2.FONT_HERSHEY_SIMPLEX,
                                fontScale=1, thickness=2, color=(255, 255, 255), lineType=cv2.LINE_AA)
    return image




start = time.perf_counter()
main()
print("Duration   : {}".format(time.perf_counter() - start))

NameError: name 'main' is not defined

In [ ]:
Ignoring emtpty camera frame.
Duration   : 12.745690459000002

Ignoring emtpty camera frame.
Duration   : 12.653819209000062


Ignoring emtpty camera frame.
Duration   : 14.740603249999936

Ignoring emtpty camera frame.
Duration   : 14.869037917000014